# **CS 363M Final Project Spring 2025**

## Chenyi Wang, Bhuvan Kannaeganti, Suyog Valsangkar

### **Overview**

For the project in this class, you will participate in a machine learning competition where you’ll apply your ML skills to a real-world dataset. You may work individually or in teams of up to 3 students. 

The dataset for this competition comes from the Austin Animal Center, the largest no-kill animal shelter in the United States. It contains historical records of animals that have entered the shelter, including details such as species, breed, age, intake type, medical condition, and other attributes. Each animal in the dataset has a recorded outcome, which represents what eventually happened to the animal after entering the shelter.

Your goal in this competition is to build a machine learning model that predicts the final outcome of each animal admitted to the shelter, based on its intake characteristics. The possible outcomes are:

**- Adopted**: The animal was placed into a new home.<br>
**- Return to Owner**: The animal was reclaimed by its original owner.<br>
**- Euthanasia**: The animal was humanely euthanized due to medical or behavioral concerns.<br>
**- Died**: The animal passed away while in the shelter’s care.<br>
**- Transfer**: The animal was moved to another shelter or rescue organization.<br>

By accurately predicting these outcomes, your model can help identify factors that influence an animal's journey through the shelter system and potentially aid in improving adoption and survival rates, shelter policies, or allocation of resources.


## **Code and Analysis Below:**

1. We need to go through the dataset and examine the existing features for patterns and methods we can feature engineer our data to enhance our final predictions. 

In [367]:
import pandas as pd

animal_data = pd.read_csv('train.csv')
animal_data.sample(5) # sample some data

,Id,Name,Intake Time,Found Location,Intake Type,Intake Condition,Animal Type,Sex upon Intake,Age upon Intake,Breed,Color,Outcome Time,Date of Birth,Outcome Type
90135,A800524,*Mick,07/22/2019 03:16:00 PM,7703 Blue Lilly Drive in Austin (TX),Stray,Normal,Cat,Intact Male,3 weeks,Domestic Shorthair,Orange Tabby/White,07/30/2019 11:02:00 AM,06/25/2019,Transfer
72004,A710141,Nadia,08/18/2015 01:49:00 PM,Tara Dr And Cats-Eye Ln in Austin (TX),Stray,Normal,Dog,Spayed Female,4 years,Tibetan Terrier/Lhasa Apso,Tricolor,08/25/2015 11:55:00 AM,08/18/2011,Transfer
37280,A719274,*Louise,01/16/2016 01:55:00 PM,710 E Ben White in Austin (TX),Stray,Normal,Dog,Intact Female,1 month,Labrador Retriever Mix,Black/White,01/21/2016 12:52:00 PM,11/16/2015,Transfer
48872,A838760,Perdita,02/17/2023 04:09:00 PM,14901 Bern Drive in Travis (TX),Stray,Injured,Dog,Spayed Female,3 years,Great Pyrenees/Border Collie,White/Black,08/02/2021 04:00:00 PM,07/12/2019,Adoption
15615,A817081,*Chimichurri,05/07/2020 11:51:00 AM,7205 Providence Avenue in Austin (TX),Stray,Sick,Dog,Intact Female,6 years,Siberian Husky,White,08/11/2020 10:53:00 AM,05/11/2013,Adoption


2. **Data Cleaning**: After observing the initial dataset, I see that we need to perform data cleaning. We need to separate the sex upon intake feature as it currently also includes sterilization health status. We also need to fix the age upon intake column as it is using all sorts of time units. The name feature also does not exist within the test set, we need to be careful and only train the model on features that exist within the test set. This also applies to Id, which does not correlate to the Id within the test set. Therefore, we can also eliminate name and id before training the model.

In [368]:
# drop name feature entirely
animal_data.drop("Name", axis=1, inplace=True)

# drop id feature entirely
animal_data.drop("Id", axis=1, inplace=True)

# drop outcome time
animal_data.drop("Outcome Time", axis=1, inplace=True)

# drop birth date
animal_data.drop("Date of Birth", axis=1, inplace=True)

# transform Animal Type into a binary column: Cat -> True, Dog -> False
animal_data['Cat'] = animal_data['Animal Type'].apply(lambda x: True if x == 'Cat' else False)
animal_data.drop('Animal Type', axis=1, inplace=True)

# separate age at intake and reproductive status, create new column
split_cols = animal_data['Sex upon Intake'].str.split(' ', n=1, expand=True)

# handle cases where the split returns "Unknown"
animal_data['Sterilized'] = split_cols[0].map({
    'Neutered': 'True',
    'Spayed': 'True',
    'Intact': 'False',
    'Unknown': 'False'
}).fillna('False')  # unexpected values, let's assume not sterilized

# assign gender, we can keep it a binary by make the feature "Male", where male = true and female = false
animal_data['Male'] = split_cols[1].apply(lambda x: False if x == 'Female' else True)
animal_data.drop("Sex upon Intake", axis=1, inplace=True)

# helper to convert age upon intake to years
def age_to_years(age_str):
    if pd.isna(age_str):
        return None
    
    number, unit = age_str.split()
    number = float(number)
    
    if "year" in unit:
        return number
    elif "month" in unit:
        return number / 12
    elif "week" in unit:
        return number / 52
    elif "day" in unit:
        return number / 365
    else:
        return None  # in case of an unexpected format

# convert age to years
animal_data['Age'] = animal_data['Age upon Intake'].apply(age_to_years)
animal_data.drop('Age upon Intake', axis=1, inplace=True)

pd.set_option('display.max_columns', None)
animal_data.sample(5) # sample some data after transformations

,Intake Time,Found Location,Intake Type,Intake Condition,Breed,Color,Outcome Type,Cat,Sterilized,Male,Age
1089,05/30/2023 02:45:00 PM,5924 S Congress Ave in Austin (TX),Abandoned,Normal,Labrador Retriever,Gold,Transfer,False,True,True,2.000000
61481,04/01/2024 01:18:00 PM,2347 Douglas Street in Austin (TX),Stray,Normal,Domestic Shorthair,Black/White,Transfer,True,False,False,0.057692
5077,07/11/2014 02:48:00 PM,2402 Burleson Ct Austin in Austin (TX),Stray,Normal,Domestic Medium Hair,Black,Transfer,True,False,False,0.096154
86246,05/04/2023 09:29:00 PM,2515 Sunny Hills Drive in Austin (TX),Stray,Normal,Australian Cattle Dog,Red Tick/White,Adoption,False,False,True,0.916667
2023,11/17/2013 12:16:00 PM,Grand Ave in Pflugerville (TX),Stray,Normal,Domestic Shorthair Mix,Black,Adoption,True,False,True,0.083333


3. **Feature Engineering**: We need to one hot encode the categorical features. However, there are categoricals that have many labels, some with very little data points. We can study the trend of their similarities to each other and group to reduce dimensionality.

Let's look at Intake Condition and see all the possible labels for that class.

In [369]:
# Count the occurrences of each intake condition label
intake_condition_counts = animal_data['Intake Condition'].value_counts()

# Print the counts for each intake condition
for condition, count in intake_condition_counts.items():
    print(f"{condition}: {count}")

Normal: 95010
Injured: 6394
Sick: 4295
Nursing: 2957
Neonatal: 1240
Aged: 373
Medical: 298
Other: 247
Pregnant: 111
Feral: 104
Med Attn: 48
Behavior: 42
Unknown: 12
Neurologic: 10
Med Urgent: 7
Parvo: 5
Space: 2
Agonal: 1
Congenital: 1


We see above that there are 19 different categorical values for the Intake Condition feature, we can merge some of these rarer classification together.

Let's explore the outcome percentages of each intake condition so we can group these conditions better and reduce the amount of labels we need to one hot encode (increases dimensionality).

In [370]:
# Iterate through each unique intake condition and calculate outcome percentages
intake_conditions = animal_data['Intake Condition'].unique()

for condition in intake_conditions:
    condition_data = animal_data[animal_data['Intake Condition'] == condition]
    total_count = len(condition_data)
    if total_count > 0:
        outcome_percentages = condition_data['Outcome Type'].value_counts(normalize=True) * 100
        print(f"Intake Condition: {condition}")
        print(outcome_percentages)
        print("-" * 50)
    else:
        print(f"Intake Condition: {condition} has no entries.")
        print("-" * 50)

Intake Condition: Normal
Outcome Type
Adoption           52.596569
Transfer           29.360067
Return to Owner    15.985686
Euthanasia          1.513525
Died                0.544153
Name: proportion, dtype: float64
--------------------------------------------------
Intake Condition: Injured
Outcome Type
Adoption           34.813888
Transfer           30.669378
Euthanasia         18.720676
Return to Owner    12.605568
Died                3.190491
Name: proportion, dtype: float64
--------------------------------------------------
Intake Condition: Pregnant
Outcome Type
Transfer           54.954955
Adoption           39.639640
Return to Owner     4.504505
Died                0.900901
Name: proportion, dtype: float64
--------------------------------------------------
Intake Condition: Neonatal
Outcome Type
Transfer           69.919355
Adoption           25.564516
Died                3.064516
Return to Owner     0.967742
Euthanasia          0.483871
Name: proportion, dtype: float64
-------

Groupings based on similar outcome percentages

In [371]:
'''
   [Group]	       [Categories]
    Normal	        Normal
    Med_Minor	    Injured, Medical (more similar outcome adopt %)
    Nursing 	    Nursing, Neonatal
    Med_Major       Med Attn, Med Urgent, Neurologic, Pregnant, Sick (more similar outcome transfer %)
    Behavioral	    Feral, Behavior
    Critical    	Agonal, Aged, Congenital, Parvo, Space, Other, Unknown
'''

'\n   [Group]\t       [Categories]\n    Normal\t        Normal\n    Med_Minor\t    Injured, Medical (more similar outcome adopt %)\n    Nursing \t    Nursing, Neonatal\n    Med_Major       Med Attn, Med Urgent, Neurologic, Pregnant, Sick (more similar outcome transfer %)\n    Behavioral\t    Feral, Behavior\n    Critical    \tAgonal, Aged, Congenital, Parvo, Space, Other, Unknown\n'

We see that our merged class labels have relatively similar outcomes percentage-wise. This will reduce dimensionality and improve our training whilst not losing the originality of the groupings.

In [372]:
# intake condition into grouped categories
condition_map = {
    'Normal': 'Normal',
    'Injured': 'Med_Minor',
    'Sick': 'Med_Major',
    'Nursing': 'Nursing',
    'Neonatal': 'Nursing',
    'Med Attn': 'Med_Major',
    'Med Urgent': 'Med_Major',
    'Medical': 'Med_Minor',
    'Neurologic': 'Med_Major',
    'Pregnant': 'Med_Major',
    'Feral': 'Behavioral',
    'Behavior': 'Behavioral',
}

# map with a fallback to 'Rare'
animal_data['Condition'] = animal_data['Intake Condition'].map(condition_map).fillna('Rare')
animal_data = pd.get_dummies(animal_data, columns=['Condition'])
animal_data.drop('Intake Condition', axis=1, inplace=True)

animal_data.sample(5)  # sample some data after transformation

,Intake Time,Found Location,Intake Type,Breed,Color,Outcome Type,Cat,Sterilized,Male,Age,Condition_Behavioral,Condition_Med_Major,Condition_Med_Minor,Condition_Normal,Condition_Nursing,Condition_Rare
27539,10/27/2021 10:25:00 AM,124 W Anderson in Austin (TX),Stray,Domestic Shorthair,Brown,Adoption,True,False,False,0.166667,False,False,False,True,False,False
35912,06/26/2014 02:22:00 PM,2504 Huntwick Dr in Austin (TX),Owner Surrender,Domestic Shorthair Mix,Brown Tabby,Euthanasia,True,True,False,9.000000,False,False,False,True,False,False
33343,08/31/2018 11:02:00 AM,Riverside And Highway 71 in Austin (TX),Stray,Labrador Retriever Mix,Black,Transfer,False,False,True,0.166667,False,False,False,True,False,False
36241,10/20/2018 06:02:00 PM,East William Cannon And Branchwood Drive in Au...,Stray,Mastiff Mix,Tan/Black,Return to Owner,False,False,False,0.416667,False,False,False,True,False,False
4326,11/05/2022 10:51:00 AM,Dessau And April Dr in Austin (TX),Stray,Flat Coat Retriever/Border Collie,Black,Transfer,False,False,True,0.500000,False,False,True,False,False,False


Let's do the same for Intake Type and check the outcome percentages of each condition.

In [373]:
# observe outcome percentages for each intake type so we can group these categoricals, reduces dimensionality
intake_types = animal_data['Intake Type'].unique()

for intake_type in intake_types:
    type_data = animal_data[animal_data['Intake Type'] == intake_type]
    total_count = len(type_data)
    if total_count > 0:
        outcome_percentages = type_data['Outcome Type'].value_counts(normalize=True) * 100
        print(f"Intake Type: {intake_type}")
        print(outcome_percentages)
        print("-" * 50)
    else:
        print(f"Intake Type: {intake_type} has no entries.")
        print("-" * 50)

Intake Type: Stray
Outcome Type
Adoption           47.774546
Transfer           34.219783
Return to Owner    13.905286
Euthanasia          3.056120
Died                1.044266
Name: proportion, dtype: float64
--------------------------------------------------
Intake Type: Public Assist
Outcome Type
Return to Owner    63.545914
Adoption           18.528788
Transfer           14.222802
Euthanasia          3.262111
Died                0.440385
Name: proportion, dtype: float64
--------------------------------------------------
Intake Type: Owner Surrender
Outcome Type
Adoption           64.671050
Transfer           26.502541
Return to Owner     5.399357
Euthanasia          2.736980
Died                0.690073
Name: proportion, dtype: float64
--------------------------------------------------
Intake Type: Abandoned
Outcome Type
Adoption           63.017032
Transfer           26.845093
Return to Owner     9.083536
Euthanasia          0.648824
Died                0.405515
Name: proportion, 

These would be the groupings that are most similar to each other based on the outcome percentages.

In [374]:
'''
    [Group]	         [Categories]
     Public Assist    Public Assist
     Stray            Stray, Wildlife
     Owner-Initiated  Abandoned, Owner Surrender
     Euthanasia       Euthanasia Request
'''

'\n    [Group]\t         [Categories]\n     Public Assist    Public Assist\n     Stray            Stray, Wildlife\n     Owner-Initiated  Abandoned, Owner Surrender\n     Euthanasia       Euthanasia Request\n'

In [375]:
# Define the mapping for grouping intake types
intake_type_map = {
    'Stray': 'Stray',
    'Public Assist': 'Public Assist',
    'Wildlife': 'Stray',
    'Abandoned': 'Owner Initiated',
    'Owner Surrender': 'Owner Initiated',
    'Euthanasia Request': 'Euthanasia'
}

# Map the intake types to their respective groups
animal_data['Intake Group'] = animal_data['Intake Type'].map(intake_type_map).fillna('Other')
animal_data = pd.get_dummies(animal_data, columns=['Intake Group'])
animal_data.drop('Intake Type', axis=1, inplace=True)

animal_data.sample(5)  # sample some data after transformation


,Intake Time,Found Location,Breed,Color,Outcome Type,Cat,Sterilized,Male,Age,Condition_Behavioral,Condition_Med_Major,Condition_Med_Minor,Condition_Normal,Condition_Nursing,Condition_Rare,Intake Group_Euthanasia,Intake Group_Owner Initiated,Intake Group_Public Assist,Intake Group_Stray
75100,04/13/2019 03:42:00 PM,11551 Maha Loop Road in Austin (TX),Staffordshire Mix,Blue/White,Adoption,False,False,False,2.0,False,False,False,True,False,False,False,False,False,True
26831,09/01/2017 02:47:00 PM,Outside Jurisdiction,Labrador Retriever Mix,Red,Transfer,False,True,True,2.0,False,False,False,True,False,False,False,False,True,False
105248,01/04/2023 06:56:00 PM,Austin (TX),Domestic Shorthair,Brown Tabby/White,Adoption,True,True,False,7.0,False,False,False,True,False,False,False,True,False,False
80077,03/03/2019 03:39:00 PM,1610 Forest Trail in Austin (TX),Domestic Shorthair Mix,Brown Tabby,Transfer,True,False,True,0.5,False,False,False,True,False,False,False,False,False,True
95884,03/31/2019 01:53:00 PM,11011 Us-290 in Manor (TX),Labrador Retriever Mix,Brown Brindle/White,Adoption,False,True,True,1.0,False,False,False,True,False,False,False,False,False,True


Still need to find a way to encode:
- Location Found
- Breed
- Color

How do they even affect the outcome or is there even a correlation? Are some colors / breeds more or less desirable than others (therefore changing adoption rates)?